In [1]:
import sympy as sp
from sympy import printing

In [2]:
# Function to compute derivatives
def compute_derivatives(function_expr, variables, lagrange=False):
    print("\n🔹 First degree partial derivatives:")
    first_order_derivatives = {var: sp.diff(function_expr, var) for var in variables}
    for var, deriv in first_order_derivatives.items():
        if lagrange:
            print(f"∂L/∂{printing.pretty(var)} = {printing.pretty(deriv)}")
        else:
            print(f"∂I/∂{printing.pretty(var)} = {printing.pretty(deriv)}")

    if not lagrange:
        # Compute and print second-order partial derivatives
        print("\n🔹 Second degree partial derivatives:")
        second_order_derivatives = {}
        for var1 in variables:
            for var2 in variables:
                second_order_derivatives[(var1, var2)] = sp.diff(first_order_derivatives[var1], var2)

        for (var1, var2), deriv in second_order_derivatives.items():
            print(f"∂²I/∂{printing.pretty(var1)}{printing.pretty(var2)} = {printing.pretty(deriv)}")

        print("\n🔹 Matrix Determinants:")
        hessian_matrix = sp.Matrix([[second_order_derivatives[(var1, var2)] for var2 in variables] for var1 in variables])
        hessian_determinants = []

        for i in range(1, len(variables) + 1):
            det_value = hessian_matrix[:i, :i].det()
            hessian_determinants.append(det_value)
            print(f"Det({i}) = {det_value}")

        # Function type detection
        print("\n🔹 Function Type:")
        if all(det > 0 for det in hessian_determinants):
            print("I -> inf.")
            return False  # Convex function -> minimization
        elif all((det < 0 if i % 2 == 0 else det >= 0) for i, det in enumerate(hessian_determinants)):
            print("I -> sup.")
            return True  # Concave function -> maximization
        else:
            print("The function is neither sup nor inf.")
            return None
    else:
        return first_order_derivatives

In [3]:
# Function to create the Lagrange function
def create_lagrange(objective, restrictions, type, variables):
    new_res = []
    rest = []

    # Process the restrictions (keep equalities intact)
    for restriction in restrictions:
        if isinstance(restriction, sp.Rel):  # Check if the restriction is a valid relational expression
            sign = restriction.rel_op
            if sign == '==':
                rest.append(restriction)  # Keep equality as is
            elif sign == '<=':
                rest.append(restriction.rhs >= restriction.lhs)  # Convert <= to >=
            elif sign == '>=':
                rest.append(restriction.rhs <= restriction.lhs)  # Convert >= to <=
        else:
            print(f"Invalid restriction: {restriction}")
            rest.append(restriction)  # Keep invalid restrictions as is

    for restriction in rest:
        lhs, rhs, sign = restriction.lhs, restriction.rhs, restriction.rel_op
        if (sign == '<=' and type == True) or (sign == '>=' and type == False):
            new_res.append(rhs - lhs)
        elif (sign == '>=' and type == True) or (sign == '<=' and type == False):
            new_res.append(lhs - rhs)
        elif sign == '==':
            new_res.append(rhs - lhs)

    lambdas = [sp.symbols(f'λ{i+1}') for i in range(len(new_res))]
    lagrange = sum(lambdas[i] * new_res[i] for i in range(len(new_res))) + objective

    print("\n🔹 Lagrange Function:")
    print(f"L{variables + lambdas} = {lagrange}")
    derives = compute_derivatives(lagrange, variables + lambdas, lagrange=True)

    index = 0
    print("\n🔹 Necessary and sufficient conditions:")
    for var, derive in derives.items():
        if var in variables:
            if rest[index].rel_op == '<=' and type or rest[index].rel_op == '>=' and not type:
                print(f"{printing.pretty(derive)} >= 0      {printing.pretty(var)}({printing.pretty(derive)}) = 0")
            else:
                print(f"{printing.pretty(derive)} <= 0      {printing.pretty(var)}({printing.pretty(derive)}) = 0")
        else:
            if rest[index].rel_op == '<=' and type or rest[index].rel_op == '>=' and not type:
                print(f"{printing.pretty(derive)} <= 0      {printing.pretty(var)}({printing.pretty(derive)}) = 0")
            elif rest[index].rel_op == '>=' and type or rest[index].rel_op == '<=' and not type:
                print(f"{printing.pretty(derive)} >= 0      {printing.pretty(var)}({printing.pretty(derive)}) = 0")
            else:
                print(f"{printing.pretty(derive)} = 0      {printing.pretty(var)}({printing.pretty(derive)}) = 0")
            index += 1

## 1

In [4]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = 8*x1 - x1**2 + 4*x2 - x2**2
restrictions = [x1 + x2 <= 2]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{printing.pretty(rest)}")
for var in [x1, x2]:
    print(f"{printing.pretty(var)} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
derivatives = create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = -x1**2 + 8*x1 - x2**2 + 4*x2 -> extr
	x₁ + x₂ ≤ 2
x₁ >= 0, x₂ >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 8 - 2⋅x₁
∂I/∂x₂ = 4 - 2⋅x₂

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = -2
∂²I/∂x₁x₂ = 0
∂²I/∂x₂x₁ = 0
∂²I/∂x₂x₂ = -2

🔹 Matrix Determinants:
Det(1) = -2
Det(2) = 4

🔹 Function Type:
I -> sup.

🔹 Lagrange Function:
L[x1, x2, λ1] = -x1**2 + 8*x1 - x2**2 + 4*x2 + λ1*(-x1 - x2 + 2)

🔹 First degree partial derivatives:
∂L/∂x₁ = -2⋅x₁ - λ₁ + 8
∂L/∂x₂ = -2⋅x₂ - λ₁ + 4
∂L/∂λ₁ = -x₁ - x₂ + 2

🔹 Necessary and sufficient conditions:
-2⋅x₁ - λ₁ + 8 <= 0      x₁(-2⋅x₁ - λ₁ + 8) = 0
-2⋅x₂ - λ₁ + 4 <= 0      x₂(-2⋅x₂ - λ₁ + 4) = 0
-x₁ - x₂ + 2 >= 0      λ₁(-x₁ - x₂ + 2) = 0


## 2

In [5]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = (x1 - 1)**2 + (x2 - 2)**2 - 3*(x1 + x2)
restrictions = [4*x1 + x2 <= 20,
                x1 + 4*x2 <= 20]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = -3*x1 - 3*x2 + (x1 - 1)**2 + (x2 - 2)**2 -> extr
	4*x1 + x2 <= 20
	x1 + 4*x2 <= 20
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 2⋅x₁ - 5
∂I/∂x₂ = 2⋅x₂ - 7

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 2
∂²I/∂x₁x₂ = 0
∂²I/∂x₂x₁ = 0
∂²I/∂x₂x₂ = 2

🔹 Matrix Determinants:
Det(1) = 2
Det(2) = 4

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = -3*x1 - 3*x2 + λ1*(4*x1 + x2 - 20) + λ2*(x1 + 4*x2 - 20) + (x1 - 1)**2 + (x2 - 2)**2

🔹 First degree partial derivatives:
∂L/∂x₁ = 2⋅x₁ + 4⋅λ₁ + λ₂ - 5
∂L/∂x₂ = 2⋅x₂ + λ₁ + 4⋅λ₂ - 7
∂L/∂λ₁ = 4⋅x₁ + x₂ - 20
∂L/∂λ₂ = x₁ + 4⋅x₂ - 20

🔹 Necessary and sufficient conditions:
2⋅x₁ + 4⋅λ₁ + λ₂ - 5 >= 0      x₁(2⋅x₁ + 4⋅λ₁ + λ₂ - 5) = 0
2⋅x₂ + λ₁ + 4⋅λ₂ - 7 >= 0      x₂(2⋅x₂ + λ₁ + 4⋅λ₂ - 7) = 0
4⋅x₁ + x₂ - 20 <= 0      λ₁(4⋅x₁ + x₂ - 20) = 0
x₁ + 4⋅x₂ - 20 <= 0      λ₂(x₁ + 4⋅x₂ - 20) = 0


## 3

In [6]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = 2*(x1**2) - 3*x1*x2 + 2*(x2**2) + 5*x1 - 6*x2
restrictions = [x1 - x2 >= 6,
                2*x1 + x2 >= 15]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = 2*x1**2 - 3*x1*x2 + 5*x1 + 2*x2**2 - 6*x2 -> extr
	x1 - x2 >= 6
	2*x1 + x2 >= 15
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 4⋅x₁ - 3⋅x₂ + 5
∂I/∂x₂ = -3⋅x₁ + 4⋅x₂ - 6

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 4
∂²I/∂x₁x₂ = -3
∂²I/∂x₂x₁ = -3
∂²I/∂x₂x₂ = 4

🔹 Matrix Determinants:
Det(1) = 4
Det(2) = 7

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = 2*x1**2 - 3*x1*x2 + 5*x1 + 2*x2**2 - 6*x2 + λ1*(-x1 + x2 + 6) + λ2*(-2*x1 - x2 + 15)

🔹 First degree partial derivatives:
∂L/∂x₁ = 4⋅x₁ - 3⋅x₂ - λ₁ - 2⋅λ₂ + 5
∂L/∂x₂ = -3⋅x₁ + 4⋅x₂ + λ₁ - λ₂ - 6
∂L/∂λ₁ = -x₁ + x₂ + 6
∂L/∂λ₂ = -2⋅x₁ - x₂ + 15

🔹 Necessary and sufficient conditions:
4⋅x₁ - 3⋅x₂ - λ₁ - 2⋅λ₂ + 5 <= 0      x₁(4⋅x₁ - 3⋅x₂ - λ₁ - 2⋅λ₂ + 5) = 0
-3⋅x₁ + 4⋅x₂ + λ₁ - λ₂ - 6 <= 0      x₂(-3⋅x₁ + 4⋅x₂ + λ₁ - λ₂ - 6) = 0
-x₁ + x₂ + 6 >= 0      λ₁(-x₁ + x₂ + 6) = 0
-2⋅x₁ - x₂ + 15 >= 0      λ₂(-2⋅x₁ - x₂ + 15) = 0


## 4

In [7]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = -2*(x1**2) - 5*(x2**2) + 3*x1 + 10*x2
restrictions = [4*x1 - x2 <= 8,
                x1 + x2 <= 12]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = -2*x1**2 + 3*x1 - 5*x2**2 + 10*x2 -> extr
	4*x1 - x2 <= 8
	x1 + x2 <= 12
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 3 - 4⋅x₁
∂I/∂x₂ = 10 - 10⋅x₂

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = -4
∂²I/∂x₁x₂ = 0
∂²I/∂x₂x₁ = 0
∂²I/∂x₂x₂ = -10

🔹 Matrix Determinants:
Det(1) = -4
Det(2) = 40

🔹 Function Type:
I -> sup.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = -2*x1**2 + 3*x1 - 5*x2**2 + 10*x2 + λ1*(-4*x1 + x2 + 8) + λ2*(-x1 - x2 + 12)

🔹 First degree partial derivatives:
∂L/∂x₁ = -4⋅x₁ - 4⋅λ₁ - λ₂ + 3
∂L/∂x₂ = -10⋅x₂ + λ₁ - λ₂ + 10
∂L/∂λ₁ = -4⋅x₁ + x₂ + 8
∂L/∂λ₂ = -x₁ - x₂ + 12

🔹 Necessary and sufficient conditions:
-4⋅x₁ - 4⋅λ₁ - λ₂ + 3 <= 0      x₁(-4⋅x₁ - 4⋅λ₁ - λ₂ + 3) = 0
-10⋅x₂ + λ₁ - λ₂ + 10 <= 0      x₂(-10⋅x₂ + λ₁ - λ₂ + 10) = 0
-4⋅x₁ + x₂ + 8 >= 0      λ₁(-4⋅x₁ + x₂ + 8) = 0
-x₁ - x₂ + 12 >= 0      λ₂(-x₁ - x₂ + 12) = 0


## 5

In [8]:
# Define variables
x1, x2, x3 = sp.symbols('x1 x2 x3')
# Define the function (Example Function)
f = x1**2 - x1*x2 + x2**2 + x3**2 - 5*x1 + 6*x2
restrictions = [4*x1 - x2 <= 8,
                x1 + x2 <= 12]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2, x3])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2, x3])


🔹 Function and restrictions:
I = x1**2 - x1*x2 - 5*x1 + x2**2 + 6*x2 + x3**2 -> extr
	4*x1 - x2 <= 8
	x1 + x2 <= 12
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 2⋅x₁ - x₂ - 5
∂I/∂x₂ = -x₁ + 2⋅x₂ + 6
∂I/∂x₃ = 2⋅x₃

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 2
∂²I/∂x₁x₂ = -1
∂²I/∂x₁x₃ = 0
∂²I/∂x₂x₁ = -1
∂²I/∂x₂x₂ = 2
∂²I/∂x₂x₃ = 0
∂²I/∂x₃x₁ = 0
∂²I/∂x₃x₂ = 0
∂²I/∂x₃x₃ = 2

🔹 Matrix Determinants:
Det(1) = 2
Det(2) = 3
Det(3) = 6

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, x3, λ1, λ2] = x1**2 - x1*x2 - 5*x1 + x2**2 + 6*x2 + x3**2 + λ1*(4*x1 - x2 - 8) + λ2*(x1 + x2 - 12)

🔹 First degree partial derivatives:
∂L/∂x₁ = 2⋅x₁ - x₂ + 4⋅λ₁ + λ₂ - 5
∂L/∂x₂ = -x₁ + 2⋅x₂ - λ₁ + λ₂ + 6
∂L/∂x₃ = 2⋅x₃
∂L/∂λ₁ = 4⋅x₁ - x₂ - 8
∂L/∂λ₂ = x₁ + x₂ - 12

🔹 Necessary and sufficient conditions:
2⋅x₁ - x₂ + 4⋅λ₁ + λ₂ - 5 >= 0      x₁(2⋅x₁ - x₂ + 4⋅λ₁ + λ₂ - 5) = 0
-x₁ + 2⋅x₂ - λ₁ + λ₂ + 6 >= 0      x₂(-x₁ + 2⋅x₂ - λ₁ + λ₂ + 6) = 0
2⋅x₃ >= 0      x₃(2⋅x₃) = 0
4⋅x₁ - x₂ - 8

## 6

In [9]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = (x1 - 1)**2 + (x2 - 2)**2
restrictions = [
    sp.Eq(-x1 + x2, 1),         # x1 + x2 = 8
    sp.Le(x1 + x2, 2)   # x1 - 2*x2 <= 16
]

print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{printing.pretty(rest)}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = (x1 - 1)**2 + (x2 - 2)**2 -> extr
	-x₁ + x₂ = 1
	x₁ + x₂ ≤ 2
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 2⋅x₁ - 2
∂I/∂x₂ = 2⋅x₂ - 4

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 2
∂²I/∂x₁x₂ = 0
∂²I/∂x₂x₁ = 0
∂²I/∂x₂x₂ = 2

🔹 Matrix Determinants:
Det(1) = 2
Det(2) = 4

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = λ1*(x1 - x2 + 1) + λ2*(x1 + x2 - 2) + (x1 - 1)**2 + (x2 - 2)**2

🔹 First degree partial derivatives:
∂L/∂x₁ = 2⋅x₁ + λ₁ + λ₂ - 2
∂L/∂x₂ = 2⋅x₂ - λ₁ + λ₂ - 4
∂L/∂λ₁ = x₁ - x₂ + 1
∂L/∂λ₂ = x₁ + x₂ - 2

🔹 Necessary and sufficient conditions:
2⋅x₁ + λ₁ + λ₂ - 2 <= 0      x₁(2⋅x₁ + λ₁ + λ₂ - 2) = 0
2⋅x₂ - λ₁ + λ₂ - 4 <= 0      x₂(2⋅x₂ - λ₁ + λ₂ - 4) = 0
x₁ - x₂ + 1 = 0      λ₁(x₁ - x₂ + 1) = 0
x₁ + x₂ - 2 <= 0      λ₂(x₁ + x₂ - 2) = 0


## 7

In [10]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = 2*x1**2 - 3*x1*x2 + 2*x2**2 + 5*x1 + 6*x2
restrictions = [sp.Eq(x1 - x2, 6),
                sp.Eq(2*x1 + x2, 15)]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = 2*x1**2 - 3*x1*x2 + 5*x1 + 2*x2**2 + 6*x2 -> extr
	Eq(x1 - x2, 6)
	Eq(2*x1 + x2, 15)
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 4⋅x₁ - 3⋅x₂ + 5
∂I/∂x₂ = -3⋅x₁ + 4⋅x₂ + 6

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 4
∂²I/∂x₁x₂ = -3
∂²I/∂x₂x₁ = -3
∂²I/∂x₂x₂ = 4

🔹 Matrix Determinants:
Det(1) = 4
Det(2) = 7

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = 2*x1**2 - 3*x1*x2 + 5*x1 + 2*x2**2 + 6*x2 + λ1*(-x1 + x2 + 6) + λ2*(-2*x1 - x2 + 15)

🔹 First degree partial derivatives:
∂L/∂x₁ = 4⋅x₁ - 3⋅x₂ - λ₁ - 2⋅λ₂ + 5
∂L/∂x₂ = -3⋅x₁ + 4⋅x₂ + λ₁ - λ₂ + 6
∂L/∂λ₁ = -x₁ + x₂ + 6
∂L/∂λ₂ = -2⋅x₁ - x₂ + 15

🔹 Necessary and sufficient conditions:
4⋅x₁ - 3⋅x₂ - λ₁ - 2⋅λ₂ + 5 <= 0      x₁(4⋅x₁ - 3⋅x₂ - λ₁ - 2⋅λ₂ + 5) = 0
-3⋅x₁ + 4⋅x₂ + λ₁ - λ₂ + 6 <= 0      x₂(-3⋅x₁ + 4⋅x₂ + λ₁ - λ₂ + 6) = 0
-x₁ + x₂ + 6 = 0      λ₁(-x₁ + x₂ + 6) = 0
-2⋅x₁ - x₂ + 15 = 0      λ₂(-2⋅x₁ - x₂ + 15) = 0


## 8

In [11]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = x1**2 - 2*x1*x2 + 2*x2**2 + 5*x1 - 6*x2
restrictions = [sp.Le(x1 + 3*x2, 12),
                sp.Ge(x1 + x2, 6)]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = x1**2 - 2*x1*x2 + 5*x1 + 2*x2**2 - 6*x2 -> extr
	x1 + 3*x2 <= 12
	x1 + x2 >= 6
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 2⋅x₁ - 2⋅x₂ + 5
∂I/∂x₂ = -2⋅x₁ + 4⋅x₂ - 6

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 2
∂²I/∂x₁x₂ = -2
∂²I/∂x₂x₁ = -2
∂²I/∂x₂x₂ = 4

🔹 Matrix Determinants:
Det(1) = 2
Det(2) = 4

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = x1**2 - 2*x1*x2 + 5*x1 + 2*x2**2 - 6*x2 + λ1*(x1 + 3*x2 - 12) + λ2*(-x1 - x2 + 6)

🔹 First degree partial derivatives:
∂L/∂x₁ = 2⋅x₁ - 2⋅x₂ + λ₁ - λ₂ + 5
∂L/∂x₂ = -2⋅x₁ + 4⋅x₂ + 3⋅λ₁ - λ₂ - 6
∂L/∂λ₁ = x₁ + 3⋅x₂ - 12
∂L/∂λ₂ = -x₁ - x₂ + 6

🔹 Necessary and sufficient conditions:
2⋅x₁ - 2⋅x₂ + λ₁ - λ₂ + 5 >= 0      x₁(2⋅x₁ - 2⋅x₂ + λ₁ - λ₂ + 5) = 0
-2⋅x₁ + 4⋅x₂ + 3⋅λ₁ - λ₂ - 6 >= 0      x₂(-2⋅x₁ + 4⋅x₂ + 3⋅λ₁ - λ₂ - 6) = 0
x₁ + 3⋅x₂ - 12 <= 0      λ₁(x₁ + 3⋅x₂ - 12) = 0
-x₁ - x₂ + 6 >= 0      λ₂(-x₁ - x₂ + 6) = 0


## 9

In [12]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = 2*x1**2 - 2*x1*x2 + 2*x2**2 + 3*x1 + 4*x2
restrictions = [sp.Ge(2*x1 + x2, 6),
                sp.Le(2*x1 + 3*x2, 18)]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = 2*x1**2 - 2*x1*x2 + 3*x1 + 2*x2**2 + 4*x2 -> extr
	2*x1 + x2 >= 6
	2*x1 + 3*x2 <= 18
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 4⋅x₁ - 2⋅x₂ + 3
∂I/∂x₂ = -2⋅x₁ + 4⋅x₂ + 4

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 4
∂²I/∂x₁x₂ = -2
∂²I/∂x₂x₁ = -2
∂²I/∂x₂x₂ = 4

🔹 Matrix Determinants:
Det(1) = 4
Det(2) = 12

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = 2*x1**2 - 2*x1*x2 + 3*x1 + 2*x2**2 + 4*x2 + λ1*(-2*x1 - x2 + 6) + λ2*(2*x1 + 3*x2 - 18)

🔹 First degree partial derivatives:
∂L/∂x₁ = 4⋅x₁ - 2⋅x₂ - 2⋅λ₁ + 2⋅λ₂ + 3
∂L/∂x₂ = -2⋅x₁ + 4⋅x₂ - λ₁ + 3⋅λ₂ + 4
∂L/∂λ₁ = -2⋅x₁ - x₂ + 6
∂L/∂λ₂ = 2⋅x₁ + 3⋅x₂ - 18

🔹 Necessary and sufficient conditions:
4⋅x₁ - 2⋅x₂ - 2⋅λ₁ + 2⋅λ₂ + 3 <= 0      x₁(4⋅x₁ - 2⋅x₂ - 2⋅λ₁ + 2⋅λ₂ + 3) = 0
-2⋅x₁ + 4⋅x₂ - λ₁ + 3⋅λ₂ + 4 <= 0      x₂(-2⋅x₁ + 4⋅x₂ - λ₁ + 3⋅λ₂ + 4) = 0
-2⋅x₁ - x₂ + 6 >= 0      λ₁(-2⋅x₁ - x₂ + 6) = 0
2⋅x₁ + 3⋅x₂ - 18 <= 0      λ₂(2⋅x₁ + 3⋅x₂ - 18) = 0


## 10

In [13]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = -x1**2 - x2**2 + x1 + 2*x2
restrictions = [sp.Le(x1 + 2*x2, 16),
                sp.Eq(x1 + x2, 8)]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = -x1**2 + x1 - x2**2 + 2*x2 -> extr
	x1 + 2*x2 <= 16
	Eq(x1 + x2, 8)
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 1 - 2⋅x₁
∂I/∂x₂ = 2 - 2⋅x₂

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = -2
∂²I/∂x₁x₂ = 0
∂²I/∂x₂x₁ = 0
∂²I/∂x₂x₂ = -2

🔹 Matrix Determinants:
Det(1) = -2
Det(2) = 4

🔹 Function Type:
I -> sup.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = -x1**2 + x1 - x2**2 + 2*x2 + λ1*(-x1 - 2*x2 + 16) + λ2*(-x1 - x2 + 8)

🔹 First degree partial derivatives:
∂L/∂x₁ = -2⋅x₁ - λ₁ - λ₂ + 1
∂L/∂x₂ = -2⋅x₂ - 2⋅λ₁ - λ₂ + 2
∂L/∂λ₁ = -x₁ - 2⋅x₂ + 16
∂L/∂λ₂ = -x₁ - x₂ + 8

🔹 Necessary and sufficient conditions:
-2⋅x₁ - λ₁ - λ₂ + 1 <= 0      x₁(-2⋅x₁ - λ₁ - λ₂ + 1) = 0
-2⋅x₂ - 2⋅λ₁ - λ₂ + 2 <= 0      x₂(-2⋅x₂ - 2⋅λ₁ - λ₂ + 2) = 0
-x₁ - 2⋅x₂ + 16 >= 0      λ₁(-x₁ - 2⋅x₂ + 16) = 0
-x₁ - x₂ + 8 = 0      λ₂(-x₁ - x₂ + 8) = 0


## 11

In [14]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = -2*x1*x2 - x1**2 - x2**2
restrictions = [sp.Le(2*x1 - x2, 6),
                sp.Le(x1 + x2, 10)]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
if type is not None:
    create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = -x1**2 - 2*x1*x2 - x2**2 -> extr
	2*x1 - x2 <= 6
	x1 + x2 <= 10
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = -2⋅x₁ - 2⋅x₂
∂I/∂x₂ = -2⋅x₁ - 2⋅x₂

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = -2
∂²I/∂x₁x₂ = -2
∂²I/∂x₂x₁ = -2
∂²I/∂x₂x₂ = -2

🔹 Matrix Determinants:
Det(1) = -2
Det(2) = 0

🔹 Function Type:
I -> sup.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2] = -x1**2 - 2*x1*x2 - x2**2 + λ1*(-2*x1 + x2 + 6) + λ2*(-x1 - x2 + 10)

🔹 First degree partial derivatives:
∂L/∂x₁ = -2⋅x₁ - 2⋅x₂ - 2⋅λ₁ - λ₂
∂L/∂x₂ = -2⋅x₁ - 2⋅x₂ + λ₁ - λ₂
∂L/∂λ₁ = -2⋅x₁ + x₂ + 6
∂L/∂λ₂ = -x₁ - x₂ + 10

🔹 Necessary and sufficient conditions:
-2⋅x₁ - 2⋅x₂ - 2⋅λ₁ - λ₂ <= 0      x₁(-2⋅x₁ - 2⋅x₂ - 2⋅λ₁ - λ₂) = 0
-2⋅x₁ - 2⋅x₂ + λ₁ - λ₂ <= 0      x₂(-2⋅x₁ - 2⋅x₂ + λ₁ - λ₂) = 0
-2⋅x₁ + x₂ + 6 >= 0      λ₁(-2⋅x₁ + x₂ + 6) = 0
-x₁ - x₂ + 10 >= 0      λ₂(-x₁ - x₂ + 10) = 0


## 12

In [15]:
# Define variables
x1, x2 = sp.symbols('x1 x2')
# Define the function (Example Function)
f = 9*(x1 -5)**2 + 4*(x2 - 6)**2
restrictions = [sp.Le(12, 3*x1 + 2*x2),
                sp.Le(x1 - x2, 6),
                sp.Le(x2, 4)]
print("\n🔹 Function and restrictions:")
print(f"I = {f} -> extr")
for rest in restrictions:
    print(f"\t{rest}")
for var in [x1, x2]:
    print(f"{var} >= 0", end=", ")
print("")
# Compute derivatives
type = compute_derivatives(f, [x1, x2])
# Create Lagrange function
create_lagrange(f, restrictions, type, [x1, x2])


🔹 Function and restrictions:
I = 9*(x1 - 5)**2 + 4*(x2 - 6)**2 -> extr
	12 <= 3*x1 + 2*x2
	x1 - x2 <= 6
	x2 <= 4
x1 >= 0, x2 >= 0, 

🔹 First degree partial derivatives:
∂I/∂x₁ = 18⋅x₁ - 90
∂I/∂x₂ = 8⋅x₂ - 48

🔹 Second degree partial derivatives:
∂²I/∂x₁x₁ = 18
∂²I/∂x₁x₂ = 0
∂²I/∂x₂x₁ = 0
∂²I/∂x₂x₂ = 8

🔹 Matrix Determinants:
Det(1) = 18
Det(2) = 144

🔹 Function Type:
I -> inf.

🔹 Lagrange Function:
L[x1, x2, λ1, λ2, λ3] = λ1*(-3*x1 - 2*x2 + 12) + λ2*(x1 - x2 - 6) + λ3*(x2 - 4) + 9*(x1 - 5)**2 + 4*(x2 - 6)**2

🔹 First degree partial derivatives:
∂L/∂x₁ = 18⋅x₁ - 3⋅λ₁ + λ₂ - 90
∂L/∂x₂ = 8⋅x₂ - 2⋅λ₁ - λ₂ + λ₃ - 48
∂L/∂λ₁ = -3⋅x₁ - 2⋅x₂ + 12
∂L/∂λ₂ = x₁ - x₂ - 6
∂L/∂λ₃ = x₂ - 4

🔹 Necessary and sufficient conditions:
18⋅x₁ - 3⋅λ₁ + λ₂ - 90 >= 0      x₁(18⋅x₁ - 3⋅λ₁ + λ₂ - 90) = 0
8⋅x₂ - 2⋅λ₁ - λ₂ + λ₃ - 48 >= 0      x₂(8⋅x₂ - 2⋅λ₁ - λ₂ + λ₃ - 48) = 0
-3⋅x₁ - 2⋅x₂ + 12 <= 0      λ₁(-3⋅x₁ - 2⋅x₂ + 12) = 0
x₁ - x₂ - 6 <= 0      λ₂(x₁ - x₂ - 6) = 0
x₂ - 4 <= 0      λ₃(x₂ - 4) = 0
